### To use this notebook, follow my process below. This focuses on doing this locally on your own machine. API will be covered later

## 0.0 Import the required packages

In [3]:
import altair as alt
import altair_viewer
import pandas as pd
import json


## 1.0. Preparing the Data for the base of the map:
   - Load the hexjson file containing the data you want to work with.
   - Make sure the hexjson file is in the correct format and accessible from the notebook.
   - You can use the `json` module to load the hexjson file into a Python dictionary.

### 1.1 Converting the JSON to a DataFrame:
   - Once you have loaded the hexjson data into a dictionary, you can convert it to a DataFrame using the `pd.DataFrame()` function from the pandas library.
   - Pass the dictionary as the argument to the `pd.DataFrame()` function to create the DataFrame.
   - Assign the resulting DataFrame to a variable for further processing.


In [66]:
alt_data = pd.read_json('/home/asia/code/willgreen93/UK_election/interface/data/uk-constituencies-2019-BBC.hexjson')
alt_df = pd.DataFrame(alt_data)


### 1.2 Cleaning up the columns
In this case, the hex file we're using is from OpenInnovations https://open-innovations.org/projects/hexmaps/constituencies/ -- check what and where to get the latitutde and longitude, for this file it will be 'r' and 'q' instead of the usual lat and lon values. You will also be able to use the 'region' key to merge your data with the hex map provided

In [83]:
## you can check values with ###
# alt_df['hexes'][0] #this is specific to this json file

alt_df[['n', 'r', 'q', 'region']] = \
    alt_df['hexes'].apply(lambda x: pd.Series([x.get('n'), x.get('r'), x.get('q'), x.get('region')]))

# #reset index to prepare for joining to results
new_df = alt_df.rename_axis('constituency_id').reset_index().drop(columns=['hexes'])


/tmp/ipykernel_11233/1721648817.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alt_df['hexes'][0] #this is specific to this json file


## 2.0 Merging the DataFrame with the Prediction (Voting Results):

If you have a separate prediction or voting results dataset that you want to merge with the main DataFrame, follow these steps:
- Load the prediction dataset into a separate DataFrame using the same process as described in step 2.
- Ensure that both DataFrames have a common column that can be used as a key for merging, you can use the constituency ID for this case
- Use the `pd.merge()` function to merge the two DataFrames based on the common column.
- Assign the merged DataFrame to a new variable for further analysis.


In [74]:
#Let's use the 2019 results (elec_data__2019.csv) here, this will be replaced \
# with the actual model predictions once we connect this to the API
results_2019 = pd.read_csv( \
    '/home/asia/code/willgreen93/UK_election/interface/data/elec_data_2019.csv')

#for the basic app, we will just use the 'incumbent_party' column and the 'constituency_id'
basic_df = results_2019[['constituency_id', 'constituency','country','incumbent_party']]

df = pd.merge(basic_df, new_df, on='constituency_id', how='left')


## 3.0 Edit params.py to fill out with the URL and the params necessary for the streamlit app